In [0]:
# Import necessary libraries
import os
from pyspark.sql import SparkSession

# Fetch the username dynamically from the Spark context
username = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()

# Fetch available catalogs dynamically
catalogs = [row[0] for row in spark.sql("SHOW CATALOGS").collect()]

# Create widgets for user input
dbutils.widgets.dropdown("catalog", catalogs[0], catalogs, "Select Catalog")
dbutils.widgets.dropdown("script_type", "Result Table Block", 
    ["Result Table Block", "Result View Block"], "Select Script Type")
dbutils.widgets.dropdown("table_type", "fact", ["fact", "dim", "hist"], "Select Table Type")
dbutils.widgets.dropdown("domain", "usg", ["usg", "evt", "ntl"], "Select Domain")
dbutils.widgets.dropdown("data_level", "CustID", ["CustID", "AgentID", "BillID"], "Select Data Level")
dbutils.widgets.dropdown("aggregation", "Day", ["Day", "Week", "Month", "Quarter", "Year"], "Select Aggregation")
dbutils.widgets.multiselect("schema_selection", "Wholesale", ["Wholesale", "Retail", "1 to 1 View"], "Select Schema Type")
dbutils.widgets.text("table_name", "MyTableName", "Enter Table Name")
dbutils.widgets.dropdown("schema_name", "EDWS", ["EDWS"], "Select Base Schema")

# Retrieve user inputs
catalog = dbutils.widgets.get("catalog")
script_type_full = dbutils.widgets.get("script_type")
table_type = dbutils.widgets.get("table_type")
domain = dbutils.widgets.get("domain")
data_level = dbutils.widgets.get("data_level")
aggregation_full = dbutils.widgets.get("aggregation")
schema_selection = dbutils.widgets.get("schema_selection")
table_name = dbutils.widgets.get("table_name")
base_schema = dbutils.widgets.get("schema_name")

# Derive abbreviations for script type and aggregation
script_type_map = {
    "Result Table Block": "R",
    "Result View Block": "V"
}
aggregation_map = {
    "Day": "D",
    "Week": "W",
    "Month": "M",
    "Quarter": "Q",
    "Year": "Y"
}
script_type = script_type_map[script_type_full]
aggregation = aggregation_map[aggregation_full]

# Generate schema names dynamically
both_schema = f"{base_schema}WR"
wholesale_view_schema = f"{base_schema}WV"
retail_view_schema = f"{base_schema}RV"
both_view_schema = f"{both_schema}V"

# Construct table name
full_table_name = f"{table_type}_{domain}_{script_type}_{data_level}_{aggregation}_{table_name}"

# Explicit columns with data types for table creation
columns = """
    cust_id INT COMMENT 'Customer ID',
    agent_id INT COMMENT 'Agent ID',
    bill_id INT COMMENT 'Bill ID',
    customer_name STRING COMMENT 'Name of the customer',
    agent_name STRING COMMENT 'Name of the agent',
    bill_amount DECIMAL(10, 2) COMMENT 'Amount of the bill',
    transaction_date TIMESTAMP COMMENT 'Date of the transaction',
    created_at TIMESTAMP COMMENT 'Record creation timestamp',
    updated_at TIMESTAMP COMMENT 'Record update timestamp'
"""

# Initialize SQL script variables
table_script = None
view_scripts = []

# Logic for Result Table Block
if script_type_full == "Result Table Block":
    # Create table in base schema(s)
    table_script = f"""
        CREATE TABLE {catalog}.{both_schema}.{full_table_name} (
            {columns}
        ) USING DELTA 
        TBLPROPERTIES ('tag' = 'mssdemo');
        """
        
    if "Wholesale" in schema_selection:
        view_scripts.append(f"""
        CREATE VIEW {catalog}.{wholesale_view_schema}.{full_table_name} AS
        SELECT * FROM {catalog}.{both_schema}.{full_table_name};
        SET TBLPROPERTIES ('tag' = 'wholesale_view');
        """)
        # Add descriptions to the columns
        view_scripts.append(f"""
        COMMENT ON VIEW {catalog}.{wholesale_view_schema}.{full_table_name} IS 'This is a description of the view';
        COMMENT ON COLUMN {catalog}.{wholesale_view_schema}.{full_table_name}.cust_id IS 'Customer ID';
        COMMENT ON COLUMN {catalog}.{wholesale_view_schema}.{full_table_name}.agent_id IS 'Agent ID';
        COMMENT ON COLUMN {catalog}.{wholesale_view_schema}.{full_table_name}.bill_id IS 'Bill ID';
        COMMENT ON COLUMN {catalog}.{wholesale_view_schema}.{full_table_name}.customer_name IS 'Name of the customer';
        COMMENT ON COLUMN {catalog}.{wholesale_view_schema}.{full_table_name}.agent_name IS 'Name of the agent';
        COMMENT ON COLUMN {catalog}.{wholesale_view_schema}.{full_table_name}.bill_amount IS 'Amount of the bill';
        COMMENT ON COLUMN {catalog}.{wholesale_view_schema}.{full_table_name}.transaction_date IS 'Date of the transaction';
        COMMENT ON COLUMN {catalog}.{wholesale_view_schema}.{full_table_name}.created_at IS 'Record creation timestamp';
        COMMENT ON COLUMN {catalog}.{wholesale_view_schema}.{full_table_name}.updated_at IS 'Record update timestamp';
        """)

    if "Retail" in schema_selection:
        view_scripts.append(f"""
        CREATE VIEW {catalog}.{retail_view_schema}.{full_table_name} AS
        SELECT * FROM {catalog}.{both_schema}.{full_table_name};
        SET TBLPROPERTIES ('tag' = 'Retail_view');
        """)
        view_scripts.append(f"""
        COMMENT ON VIEW {catalog}.{retail_view_schema}.{full_table_name} IS 'This is a description of the view';
        COMMENT ON COLUMN {catalog}.{retail_view_schema}.{full_table_name}.cust_id IS 'Customer ID';
        COMMENT ON COLUMN {catalog}.{retail_view_schema}.{full_table_name}.agent_id IS 'Agent ID';
        COMMENT ON COLUMN {catalog}.{retail_view_schema}.{full_table_name}.bill_id IS 'Bill ID';
        COMMENT ON COLUMN {catalog}.{retail_view_schema}.{full_table_name}.customer_name IS 'Name of the customer';
        COMMENT ON COLUMN {catalog}.{retail_view_schema}.{full_table_name}.agent_name IS 'Name of the agent';
        COMMENT ON COLUMN {catalog}.{retail_view_schema}.{full_table_name}.bill_amount IS 'Amount of the bill';
        COMMENT ON COLUMN {catalog}.{retail_view_schema}.{full_table_name}.transaction_date IS 'Date of the transaction';
        COMMENT ON COLUMN {catalog}.{retail_view_schema}.{full_table_name}.created_at IS 'Record creation timestamp';
        COMMENT ON COLUMN {catalog}.{retail_view_schema}.{full_table_name}.updated_at IS 'Record update timestamp';
        """)

    if "1 to 1 View" in schema_selection:
        view_scripts.append(f"""
        CREATE VIEW {catalog}.{both_view_schema}.{full_table_name} AS
        SELECT * FROM {catalog}.{both_schema}.{full_table_name};
        SET TBLPROPERTIES ('tag' = '1 to 1_view');
        """)

# Logic for Result View Block
elif script_type_full == "Result View Block":
    # Create views only
    if "Wholesale" in schema_selection:
        view_scripts.append(f"""
        CREATE VIEW {catalog}.{wholesale_view_schema}.{full_table_name} AS
        SELECT 
            cust_id, agent_id, bill_id, customer_name, agent_name, 
            bill_amount, transaction_date, created_at, updated_at
        FROM {catalog}.{both_schema}.{full_table_name};
        """)
    if "Retail" in schema_selection:
        view_scripts.append(f"""
        CREATE VIEW {catalog}.{retail_view_schema}.{full_table_name} AS
        SELECT 
            cust_id, agent_id, bill_id, customer_name, agent_name, 
            bill_amount, transaction_date, created_at, updated_at
        FROM {catalog}.{both_schema}.{full_table_name};
        """)
    if "1 to 1 View" in schema_selection:
        view_scripts.append(f"""
        CREATE VIEW {catalog}.{both_view_schema}.{full_table_name} AS
        SELECT 
            cust_id, agent_id, bill_id, customer_name, agent_name, 
            bill_amount, transaction_date, created_at, updated_at
        FROM {catalog}.{both_schema}.{full_table_name}
        UNION ALL
        SELECT 
            cust_id, agent_id, bill_id, customer_name, agent_name, 
            bill_amount, transaction_date, created_at, updated_at
        FROM {catalog}.{both_schema}.{full_table_name};
        """)
# Print the table script and view scripts
if table_script:
    print("Table Creation Script:")
    print(table_script)

if view_scripts:
    print("View Creation Scripts:")
    for script in view_scripts:
        print(script)
        
template_path = f"/dbfs/tmp/script_templates/Poojitha/scripts.txt"

# Initialize the content of the file with the table script and view scripts
script_content = ""

# If the table_script exists, add it to the file content
if table_script:
    script_content += f"Table Creation Script:\n{table_script}\n\n"

# If the view_scripts exist, add them to the file content
if view_scripts:
    script_content += "View Creation Scripts:\n"
    for script in view_scripts:
        script_content += f"{script}\n"

if script_content:
    dbutils.fs.put(template_path,script_content,overwrite=True)
    print(f"Scripts saved to: {template_path}")
else:
    print("No scripts to save.")